In [16]:
import numpy as np
import pandas as pd
import cv2
from skimage import io

In [17]:
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub


In [18]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

In [19]:
m = hub.KerasLayer('https://tfhub.dev/google/aiy/vision/classifier/food_V1/1')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [20]:
inputFood_url = "foodimage2.jpeg"
labelmap_url = "aiy_food_V1_labelmap.csv"
input_shape = (224, 224)


In [21]:
image = np.asarray(io.imread(inputFood_url), dtype="float")
image = cv2.resize(image, dsize=input_shape, interpolation=cv2.INTER_CUBIC)
# Scale values to [0, 1].
image = image / image.max()
image = image.astype(np.float32)
# The model expects an input of (?, 224, 224, 3).
images = np.expand_dims(image, 0)

In [22]:
output = m(images)
predicted_index = output.numpy().argmax()
classes = list(pd.read_csv(labelmap_url)["name"])
print("Prediction: ", classes[predicted_index])

Prediction:  Waffle


In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(m)
tflite_model = converter.convert()
with open('food_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/lk/7j1rwkqx2w1f1m__90437jbc0000gn/T/tmpt8vpdkvj/assets


INFO:tensorflow:Assets written to: /var/folders/lk/7j1rwkqx2w1f1m__90437jbc0000gn/T/tmpt8vpdkvj/assets
2023-04-05 12:40:46.307824: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-04-05 12:40:46.308014: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.


In [25]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from skimage import io

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="food_classifier.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load the image to be classified.
image_url = "foodImage.jpeg"
image = np.asarray(io.imread(image_url), dtype="float")
image = cv2.resize(image, dsize=input_shape, interpolation=cv2.INTER_CUBIC)
image = image / image.max()
image = np.expand_dims(image, 0)

# Set the input tensor.
interpreter.set_tensor(input_details[0]['index'], image)

# Invoke the interpreter.
interpreter.invoke()

# Get the output tensor.
output = interpreter.get_tensor(output_details[0]['index'])
predicted_index = np.argmax(output)
classes = list(pd.read_csv(labelmap_url)["name"])
print("Prediction: ", classes[predicted_index])


ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_images:0 